In [3]:
import sys
sys.path.append(r'D:\Pixta\ultralytics_custom5')
import pandas as pd
import json
import os
import cv2
import torch
from ultralytics import YOLO
from matplotlib import pyplot as plt
import time
import numpy as np

In [4]:
info_label = {
    'age': ['20-30s', '40-50s', 'Baby', 'Kid', 'Senior', 'Teenager'],
    'race': ['Caucasian', 'Mongoloid', 'Negroid'],
    'masked': ['masked', 'unmasked'],
    'skintone': ['dark', 'light', 'mid-dark', 'mid-light'],
    'emotion': ['Anger', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sadness', 'Surprise'],
    'gender': ['Female', 'Male']
}

In [5]:
def imshow_np_image(img):
    plt.imshow(img[:,:,::-1])    
    plt.show()

def tensor_to_float(lis):
    # lis = [int(np.round(i.cpu().detach().numpy()) for i in lis)]
    lis = [float(i.cpu().detach().numpy()) for i in lis]
    return lis
    
def process_out_detect(img, results):
    bbox_lis = []
    for r in results:
        boxes = r.boxes
        for i, box in enumerate(boxes):
            x, y, w, h = box.xywh[0]

            bbox_lis.append(tensor_to_float([x-w/2, y-h/2, w, h]))

            cv2.rectangle(img, (int(x-w/2),int(y-h/2)), (int(x+w/2),int(y+h/2)), (0,255,0), 3)
            # cv2.putText(img, str(i+1) ,(x1,y1+20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 1)
    return bbox_lis

def sample_values(lis):
    print(lis)
    lis = list(lis)
    while len(lis) < 30:
        lis.append(-1)
    return lis

def convert_to_text(key, index):
    return info_label[key][index]

def procees_out_cls(results, index_box):
    preds = {}
    num_cate = iter([6,3,2,4,7,2])
    for key in results.keys():
        cur_cate = next(num_cate)
        pred = results[key]
        pred = pred[:, (index_box * cur_cate):((index_box + 1) * cur_cate)]
        pred = torch.softmax(pred, dim= -1)
        pred = torch.argmax(pred, dim= -1)
        index = int(pred.cpu().detach().numpy())
        preds[key] = convert_to_text(key, index)
    return preds

In [14]:
image_id = json.load(open('file_name_to_image_id_preal.json'))

In [17]:
# path = '../../private_test_data/private_test_data/'
# names = os.listdir(path)
# np.random.shuffle(names)
# for index, name in enumerate(image_id.keys()):
#     if index < 10:
#         continue
#     id = image_id[name]
#     print(id)
#     img = cv2.imread(path + name)
#     bbox = image_id['annotations'][index]['bbox']
#     bbox = str(bbox)
#     bbox = np.array(bbox[1:-1].split(", "), dtype=np.float32)
#     bbox = bbox.astype(np.int16)
#     image = cv2.rectangle(img, (bbox[0], bbox[1]), (bbox[0] + bbox[2], bbox[1] + bbox[3]), (255, 255, 0), 2)
#     plt.imshow(img[:,:,::-1])
#     break

11


TypeError: string indices must be integers

In [ ]:
# path = 'private_test/data/'
# names = os.listdir(path)
# # np.random.shuffle(names)

# label = pd.DataFrame(columns= ['id', 'image_id', 'category_id', 'bbox', 'area', 'iscrowd', 'skintone', 'age', 'race', 'emotion', 'gender', 'masked'])
# for index, name in enumerate(image_id['images']):
#     dic = image_id['annotations'][index]
#     print(dic.keys())
#     label.loc[index] = [dic[key] for key in dic.keys()]
#     break

# label.to_csv('answer.csv', index= False)
# label.head()

dict_keys(['id', 'image_id', 'category_id', 'bbox', 'area', 'iscrowd', 'skintone', 'age', 'race', 'emotion', 'gender', 'masked'])


,id,image_id,category_id,bbox,area,iscrowd,skintone,age,race,emotion,gender,masked
0,1,1,0,"[584.1895944369563, 301.32785213219023, 265.74...",86512.245521,0,mid-light,20-30s,Mongoloid,Anger,Male,unmasked


In [21]:
yolo = YOLO('../pretrained\yolov8s\\pbest.pt')
img = cv2.imread('../public_test/100147591.jpg')
a = yolo(img)


0: 224x320 1 face, 271.3ms
Speed: 8.0ms preprocess, 271.3ms inference, 8.4ms postprocess per image at shape (1, 3, 224, 320)


In [ ]:
path = '../../private_test_data/private_test_data/'
all_preds = []
times = []
names = os.listdir(path)
for index, name in enumerate(image_id):
    dic = image_id['annotations'][index]
    print(index)
    name_id = image_id[name]
    s = time.time()
    results = yolo(path + name)
    e = time.time()
    times.append(e-s)

    bbox_lis = []
    area= []
    for r in results[0]:
        boxes = r.boxes
        for i, box in enumerate(boxes):
            x, y, w, h = box.xywh[0]
            bbox_lis.append(tensor_to_float([x-w/2, y-h/2, w, h]))
            area.append(int(w*h))

    preds = {}
    is_crowd= 0
    for i in range(len(bbox_lis)):
        num_cate = iter([6,3,2,4,7,2])
        for key in results[1].keys():
            cur_cate = next(num_cate)
            pred = results[1][key]
            pred = pred[:, (i * cur_cate):((i + 1) * cur_cate)]
            pred = torch.softmax(pred, dim= -1)
            pred = torch.argmax(pred, dim= -1)
            index = int(pred.cpu().detach().numpy())
            preds[key] = convert_to_text(key, index)
        
        all_preds.append([dic['id'], dic['image_id'], dic['category_id'],bbox_lis[i],area[i], is_crowd, preds['skintone'], preds['age'], preds['race'], preds['emotion'], preds['gender'], preds['masked']])
        break

    


0: 320x256 2 faces, 302.1ms
Speed: 3.1ms preprocess, 302.1ms inference, 3.0ms postprocess per image at shape (1, 3, 320, 256)

0: 224x320 1 face, 35.7ms
Speed: 3.0ms preprocess, 35.7ms inference, 2.0ms postprocess per image at shape (1, 3, 224, 320)

0: 256x320 1 face, 149.7ms
Speed: 2.0ms preprocess, 149.7ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 320)

0: 224x320 1 face, 36.9ms
Speed: 2.0ms preprocess, 36.9ms inference, 2.0ms postprocess per image at shape (1, 3, 224, 320)

0: 256x320 1 face, 37.2ms
Speed: 1.5ms preprocess, 37.2ms inference, 1.6ms postprocess per image at shape (1, 3, 256, 320)

0: 224x320 1 face, 37.2ms
Speed: 5.0ms preprocess, 37.2ms inference, 2.0ms postprocess per image at shape (1, 3, 224, 320)

0: 192x320 1 face, 245.7ms
Speed: 2.0ms preprocess, 245.7ms inference, 3.0ms postprocess per image at shape (1, 3, 192, 320)

0: 224x320 2 faces, 40.1ms
Speed: 3.5ms preprocess, 40.1ms inference, 34.6ms postprocess per image at shape (1, 3, 224, 320)

In [ ]:
all_preds[0]

[1,
 1,
 0,
 [586.53466796875, 287.4141845703125, 260.3707275390625, 336.8221435546875],
 87698,
 0,
 'mid-dark',
 '20-30s',
 'Negroid',
 'Neutral',
 'Female',
 'masked']

In [ ]:
# file_name,bbox,image_id,race,age,emotion,gender,skintone,masked
final = pd.DataFrame(columns= ['id', 'image_id', 'category_id', 'bbox', 'area', 'iscrowd', 'skintone', 'age', 'race', 'emotion', 'gender', 'masked'], data= all_preds)

In [ ]:
final

,id,image_id,category_id,bbox,area,iscrowd,skintone,age,race,emotion,gender,masked
0,1,1,0,"[586.53466796875, 287.4141845703125, 260.37072...",87698,0,mid-dark,20-30s,Negroid,Neutral,Female,masked
1,2,2,0,"[1130.552001953125, 312.98077392578125, 290.54...",110847,0,dark,20-30s,Negroid,Neutral,Female,masked
2,3,3,0,"[554.548095703125, 275.7889404296875, 235.3370...",49822,0,dark,20-30s,Negroid,Neutral,Female,masked
3,4,4,0,"[899.781982421875, 69.75421142578125, 169.1840...",41346,0,dark,20-30s,Negroid,Neutral,Female,masked
4,5,5,0,"[865.802734375, 479.9620361328125, 208.5083007...",58104,0,mid-dark,20-30s,Negroid,Happiness,Female,masked
...,...,...,...,...,...,...,...,...,...,...,...,...
12637,12649,12564,0,"[214.74920654296875, 179.26803588867188, 591.0...",450426,0,mid-dark,20-30s,Negroid,Neutral,Female,masked
12638,12650,12565,0,"[172.03680419921875, 153.38735961914062, 593.9...",471167,0,mid-dark,20-30s,Negroid,Neutral,Female,masked
12639,12651,12566,0,"[243.54852294921875, 185.29733276367188, 543.0...",407961,0,mid-dark,20-30s,Negroid,Neutral,Female,masked
12640,12652,12567,0,"[200.4273681640625, 162.22769165039062, 585.42...",459697,0,mid-dark,20-30s,Negroid,Neutral,Female,masked


In [ ]:
final.to_csv('answer.csv', index= False)